In [ ]:
# !pip install ogame

In [ ]:
# https://github.com/alaingilbert/pyogame/wiki/Code-Snippets

In [26]:
import os
import sys
module_path = os.path.abspath(os.path.join('../pyogame'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ogame import OGame, constants
import time
import re

In [27]:
o = OGame(
    universe="Ymir",
    # universe='Volans',
    username="samchakerian@gmail.com",
    password="$R3e@W1q1q"
)

In [28]:
id = o.planet_ids()[0]
supply = o.supply(id)
res = o.resources(id)

In [29]:
# check bldg construction
{
    'sp': supply.solar_plant.in_construction,
    'mm': supply.metal_mine.in_construction,
    'cm': supply.crystal_mine.in_construction,
    'dm': supply.deuterium_mine.in_construction
}

{'sp': False, 'mm': False, 'cm': False, 'dm': False}

In [30]:
# is possible
{
    'sp': supply.solar_plant.is_possible,
    'mm': supply.metal_mine.is_possible,
    'cm': supply.crystal_mine.is_possible,
    'dm': supply.deuterium_mine.is_possible
}

{'sp': True, 'mm': True, 'cm': True, 'dm': True}

In [31]:
# check resources
{
    'sp': res.energy,
    'mm': res.metal,
    'cm': res.crystal,
    'dm': res.deuterium
}

{'sp': 418, 'mm': 68704, 'cm': 22474, 'dm': 61418}

In [32]:
# check levels
{
    'sp': supply.solar_plant.level,
    'mm': supply.metal_mine.level,
    'cm': supply.crystal_mine.level,
    'dm': supply.deuterium_mine.level
}

{'sp': 14, 'mm': 10, 'cm': 8, 'dm': 8}

In [33]:
# to request esp reports

from bs4 import BeautifulSoup

def BeautifulSoup4(response):
    parsed = BeautifulSoup(response, features="html5lib")

    def find_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find(**kwargs)

    def find_all_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find_all(**kwargs)

    parsed.find_partial = find_partial
    parsed.find_all_partial = find_all_partial
    return parsed

In [34]:
def get_reports():
    
    response = o.session.get(
        url=o.index_php,
        params={'page': 'messages',
                'tab': 20,
                'ajax': 1}
    ).text
    bs4 = BeautifulSoup4(response)
    report_links = [
        link['href']
        for link in bs4.find_all_partial(href='page=messages&messageId')
    ]
    
    data = []
    for link in report_links:
        response = o.session.get(link).text
        bs4 = BeautifulSoup4(response)
        data.append({'link': link, 'text': bs4})
        
    return data

In [35]:
def clean_reports(data):
    
    for report in data:
        rep_res = [_.text for _ in report['text'].find_all('span', class_="res_value")]
        rep_coord = report['text'].find('span', class_="msg_title new blue_txt").text
        try:
            rep_def = [_.text for _ in report['text'].find('ul', attrs={"data-type": "defense"}).find('li').find_all('span')]
        except:
            rep_def = None
        report['results'] = {
            'res': rep_res,
            'rep_coord': rep_coord,
            'rep_def': rep_def
        }
        
    return [_['results'] for _ in data]

In [41]:
# for planet in o.galaxy(constants.coordinates(1, 447)):
#     print(planet.status)

# Iterate thru galaxies
# Cut off after 10
# Check the reports, append data (cleaned)
# Continue with other 10, until done
#

data = []

r = 20
h = 447
s = h-r
e = h+r
if e > 499:
    e = 499
    
counter = 0
for galaxy in range(s, e):
    for planet in o.galaxy(constants.coordinates(1, galaxy)):
        
        # stop, check reports
        if counter >= 9:
            temp_data = get_reports()
            temp_data = clean_reports(temp_data)
            data.extend(temp_data)
            counter = 0
            time.sleep(5)
        
        # send the probes
        if 'inactive' in planet.status and 'vacation' not in planet.status:
            send = o.send_fleet(
                constants.mission.spy, 
                o.planet_ids()[0], 
                planet.position, 
                [constants.ships.espionage_probe(1)]
            )
            counter+=1
            print(galaxy, planet.status, planet.name, planet.rank, send)

437 ['inactive'] Homeworld 1162 True
439 ['inactive'] Homeworld 1718 True
439 ['inactive'] Homeworld 1560 True
439 ['inactive'] Homeworld 1346 True
440 ['inactive'] Homeworld 1634 True
441 ['inactive'] Homeworld 1462 True
444 ['inactive'] Homeworld 1438 True
444 ['inactive'] Howak 1121 True
445 ['inactive'] Homeworld 1348 False
445 ['inactive'] Homeworld 1136 True
448 ['inactive'] Homeworld 1579 True
449 ['inactive'] Homeworld 1635 True
450 ['inactive'] Beer 565 True
451 ['inactive'] Homeworld 1531 True
451 ['inactive'] Homeworld 1532 True
452 ['inactive'] Homeworld 1258 True
453 ['inactive'] Homeworld 1109 True


In [46]:
# find how many res you can get:
coords_and_res = []
for _ in data:
    if _['rep_def'] == None:
        res_clean = [int(__.replace('.', '')) for __ in _['res'][:3]]
        coords_and_res.append([sum(res_clean), [int(_) for _ in re.search(r'[0-9:]{1,}', _['rep_coord']).group(0).split(":")]])

In [50]:
sorted(coords_and_res, reverse=True)

[[24247, [1, 439, 4]],
 [20013, [1, 439, 6]],
 [20000, [1, 439, 12]],
 [16906, [1, 452, 6]],
 [11669, [1, 449, 12]],
 [6014, [1, 451, 12]],
 [5364, [1, 451, 6]]]

In [51]:
# get ships
ships = o.ships(id)

In [60]:
# deploy ships

lcs = ships.large_transporter.amount
scs = ships.small_transporter.amount

for _ in sorted(coords_and_res, reverse=True)[1:]:

    coords = _[1]
    
    if _[0]/2 > 25000 and lcs > 0:
        n = int(_[0]/2/25000)
        to_send = constants.ships.large_transporter(n)
        lcs -= n
        
    else:
        n = int(_[0]/2/5000)
        to_send = constants.ships.small_transporter(n)
        scs -= n

    send = o.send_fleet(
        constants.mission.attack,
        o.planet_ids()[0],
        (coords[0], coords[1], coords[2], 1),
        [to_send]
    )
    print(_, to_send, send)
    print('remaining large:', lcs, 'remaining small:', scs)

[20013, [1, 439, 6]] (202, 2, 'shipyard') True
remaining large: 9 remaining small: 10
[20000, [1, 439, 12]] (202, 2, 'shipyard') True
remaining large: 9 remaining small: 8
[16906, [1, 452, 6]] (202, 1, 'shipyard') True
remaining large: 9 remaining small: 7
[11669, [1, 449, 12]] (202, 1, 'shipyard') True
remaining large: 9 remaining small: 6
[6014, [1, 451, 12]] (202, 0, 'shipyard') False
remaining large: 9 remaining small: 6
[5364, [1, 451, 6]] (202, 0, 'shipyard') False
remaining large: 9 remaining small: 6
